In [1]:
# Import required libraries
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy

# Initialize Flask app
app = Flask(__name__)

# Configure SQLite database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///canteen_orders.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

# Define Order model
class Order(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    customer_name = db.Column(db.String(100), nullable=False)
    item = db.Column(db.String(100), nullable=False)
    quantity = db.Column(db.Integer, nullable=False)

    def __repr__(self):
        return f"<Order {self.customer_name} ordered {self.quantity} x {self.item}>"

# Create database tables
@app.before_first_request
def create_tables():
    db.create_all()

# API endpoint to place orders
@app.route('/order', methods=['POST'])
def place_order():
    data = request.get_json()

    # Basic validation
    if not all(k in data for k in ("customer_name", "item", "quantity")):
        return jsonify({"error": "Missing fields in request"}), 400

    try:
        order = Order(
            customer_name=data['customer_name'],
            item=data['item'],
            quantity=int(data['quantity'])
        )
        db.session.add(order)
        db.session.commit()
        return jsonify({"message": "Order placed successfully"}), 201
    except Exception as e:
        return jsonify({"error": str(e)}), 500

# API endpoint to view all orders
@app.route('/orders', methods=['GET'])
def get_orders():
    orders = Order.query.all()
    result = [{
        "id": order.id,
        "customer_name": order.customer_name,
        "item": order.item,
        "quantity": order.quantity
    } for order in orders]
    return jsonify(result), 200

# Run the Flask app
if __name__ == '__main__':
    app.run(debug=True, port=5001)


ModuleNotFoundError: No module named 'flask_sqlalchemy'